## Preprocessing

In [1]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.7 MB/s eta 0:00:00


In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
application_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   EIN                     34299 non-null  int64 
 1   NAME                    34299 non-null  object
 2   APPLICATION_TYPE        34299 non-null  object
 3   AFFILIATION             34299 non-null  object
 4   CLASSIFICATION          34299 non-null  object
 5   USE_CASE                34299 non-null  object
 6   ORGANIZATION            34299 non-null  object
 7   STATUS                  34299 non-null  int64 
 8   INCOME_AMT              34299 non-null  object
 9   SPECIAL_CONSIDERATIONS  34299 non-null  object
 10  ASK_AMT                 34299 non-null  int64 
 11  IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 3.1+ MB


In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop('EIN', axis=1)
application_df = application_df.drop('NAME', axis=1)
application_df = application_df.drop('STATUS', axis=1)
application_df = application_df.drop('ASK_AMT', axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,N,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,N,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,N,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,N,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,N,1


In [5]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
IS_SUCCESSFUL              2
dtype: int64

In [6]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [7]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
#application_types_to_replace = ['T9', 'T13', 'T12','T2','T25', 'T14', 'T29', 'T15', 'T17']
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts().loc[lambda x : x<500].index.values
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts().loc[lambda x : x>1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = application_df['CLASSIFICATION'].value_counts().loc[lambda x : x<1500].index.values

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [10]:
application_df.nunique()

APPLICATION_TYPE          9
AFFILIATION               6
CLASSIFICATION            6
USE_CASE                  5
ORGANIZATION              4
INCOME_AMT                9
SPECIAL_CONSIDERATIONS    2
IS_SUCCESSFUL             2
dtype: int64

In [11]:
application_df.dtypes

APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
IS_SUCCESSFUL              int64
dtype: object

In [12]:
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,N,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,N,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,N,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,N,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,N,1


In [13]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)
application_df.head()

,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,1,False,False,False,True,False,False,False,False,False,...,True,False,False,False,False,False,False,False,True,False
2,0,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False
3,1,False,False,False,True,False,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False
4,1,False,False,False,True,False,False,False,False,False,...,False,False,True,False,False,False,False,False,True,False


In [14]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(columns='IS_SUCCESSFUL').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [182]:
 # Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=2))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 50)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

## Compile, Train and Evaluate the Model

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
#number_input_features = X_train.iloc[0].shape[0]
hidden_nodes_layer1 =  100
hidden_nodes_layer2 = 20
#hidden_nodes_layer3 = 7

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# third hidden layer#
#nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 100)               4200      
                                                                 
 dense_4 (Dense)             (None, 20)                2020      
                                                                 
 dense_5 (Dense)             (None, 1)                 21        
                                                                 
Total params: 6241 (24.38 KB)
Trainable params: 6241 (24.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50)

Epoch 1/50
804/804 [==============================] - 3s 2ms/step - loss: 0.5698 - accuracy: 0.7217
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5553 - accuracy: 0.7287
Epoch 3/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5529 - accuracy: 0.7295
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5508 - accuracy: 0.7299
Epoch 5/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5501 - accuracy: 0.7295
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5488 - accuracy: 0.7304
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5482 - accuracy: 0.7324
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5467 - accuracy: 0.7338
Epoch 9/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5466 - accuracy: 0.7326
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5463 - accuracy: 0.7328

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5528 - accuracy: 0.7256 - 516ms/epoch - 2ms/step
Loss: 0.5527815818786621, Accuracy: 0.7255976796150208


In [21]:
# Define the 2nd model attempt - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  7
hidden_nodes_layer2 = 3

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 7)                 294       
                                                                 
 dense_7 (Dense)             (None, 3)                 24        
                                                                 
 dense_8 (Dense)             (None, 1)                 4         
                                                                 
Total params: 322 (1.26 KB)
Trainable params: 322 (1.26 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [23]:
# Train the model
fit_model = nn2.fit(X_train_scaled,y_train,epochs=50)

Epoch 1/50
804/804 [==============================] - 4s 3ms/step - loss: 0.6415 - accuracy: 0.6761
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5759 - accuracy: 0.7234
Epoch 3/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5638 - accuracy: 0.7266
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5586 - accuracy: 0.7284
Epoch 5/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5568 - accuracy: 0.7291
Epoch 6/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5560 - accuracy: 0.7290
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5551 - accuracy: 0.7298
Epoch 8/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5545 - accuracy: 0.7305
Epoch 9/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5539 - accuracy: 0.7301
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5535 - accuracy: 0.7308

In [29]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5526 - accuracy: 0.7272 - 711ms/epoch - 3ms/step
Loss: 0.5526029467582703, Accuracy: 0.7272303104400635


In [43]:
# Define the 3rd model attempt - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  5
hidden_nodes_layer2 = 2

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="softmax")
)

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 5)                 210       
                                                                 
 dense_19 (Dense)            (None, 2)                 12        
                                                                 
 dense_20 (Dense)            (None, 1)                 3         
                                                                 
Total params: 225 (900.00 Byte)
Trainable params: 225 (900.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [44]:
# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [46]:
# Train the model
fit_model = nn3.fit(X_train_scaled,y_train,epochs=50)

Epoch 1/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5527 - accuracy: 0.7301
Epoch 2/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5525 - accuracy: 0.7313
Epoch 3/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5525 - accuracy: 0.7306
Epoch 4/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5523 - accuracy: 0.7306
Epoch 5/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5521 - accuracy: 0.7305
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5517 - accuracy: 0.7323
Epoch 7/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5519 - accuracy: 0.7315
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5520 - accuracy: 0.7310
Epoch 9/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5518 - accuracy: 0.7314
Epoch 10/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5517 - accuracy: 0.7313

In [47]:
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5551 - accuracy: 0.7307 - 455ms/epoch - 2ms/step
Loss: 0.5550943613052368, Accuracy: 0.7307288646697998


In [42]:
# Export our model to HDF5 file

nn3.save("nn3.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
